# Hyperparameter Optimization 

## 1.1 소개

* AutoML이란 performance 최적화를 위해서 hyperparameter를 자동으로 적절히 설정하는 것이다. 
* 왜 이런 자동화가 필요한다. 
  * 사람의 수고를 덜기 위해서. Huamn-out-of-loop
  * 성능을 개선하기 위해 => led to SOTA
  * 재현성 향상을 위해
    * 다른 사람 알고리즘만 주어지면 AutoML로써 극대화 성능 재현 
* HPO는 왜 어려운 문제인가?
  * function evaluation에 시간이 많이 소요된다. 
  * configuration space가 너무 복잡하다 
    * 범위도 광대하고, 연속/이산이 섞여있고, 매우 고차원이고..
  * 보통은 gradient w.r.t hyperparameter 정보를 얻을 수 없다. (즉 blackbox)
  * 학습셋 크기가 제한적이라서 직접적으로 일반화 성능 최적화를 할 수 없다. 


## 1.2 문제 정의 

* N개의 hyperparameter가 있으면 N-sized vector of hyperparameter lambda로 표현
* A가 특정 알고리즘(ex. NN)이라면  A_lambda는 해당 hyperparaneter로 셋팅된 모델

### hyperparameter 도메인 특징
* 타입이 다양하다. 
  * real (ex. learning rate)
  * integer (ex. # of layers)
  * binary (ex. use of dropout or not)
  * categorical (ex. types of optimizer)
* conditional하다. 
  * ex) 
  
### Hyperparameter 최적화 목표

![](./images/ch01/01.png)


* validation protocol은 다양하다. 
  * 분류 문제라면 holdout set에 대해서 accuracy

#### validation protocol 수행 시간 감소 전략 

* only ubset of folds, only subset of data
* for a small amount of iterations
* multi-task, multi-score, auxiliary task

  

## 1.2.2 다중 목표 최적화 
* 때로는 두개 이상의 목표가 trade-off 관계일 수 있다. 
  * ex) 모델 성능과 필요한 데이터량
* 다중 목표에 대한 최적화 방법 두 가지 
  * 두 번째 목표를 constraint로 바꾸고, 제약 최적화(constrainted optimization)으로 접근한다. 
  * 파레토 프론트를 찾는 문제로 본다. 

## 1.3 Blackbox HPO

### Model-free B-HPO

* Grid Search v.s Random Search

![](./images/ch01/02.png )

### Grid Search는 별로다

* 차원의 저주에 시달림 ex) 2^N for binary domain
* 더 촘촘히 격자를 짜면 더 많은 function 평가를 해야 함

### Random Search가 더 낫다

* 특히 hyperparameter별로 중요도가 다른 경우 더 빛을 발함
* worker들간에 소통할 일이 없으니 병렬화에 좋다. 
* 그래서 HPO에서는 일종의 baseline으로 사용을 하면 된다. 
* model-based 방법과 interleave하게 사용하면 일종의 exploration 효과를 주는 셈일 것이다. 

### Random Search는 Guided Search보다는 못하다.

* Guided search를 예를 들어, BO는 N번의 함수 수행으로 optimum에 근접하지만
* Random search는 평균적으로 2^N 함수 수행해야 optimum에 근접

### Popluation-based 방법
* such as genetic algorithm
* CMA-ES


## 1.4 Multi-fidelity Optimization

* Multifidelity simulation
  * ultifidelity methods leverage both low- and high-fidelity data in order to maximize the accuracy of model estimates, while minimizing the cost

![](./images/ch01/03.png )

* 필요성 
  * HPO에서는 함수평가를 수없이 해야 하는데 데이터가 많아질수록, 모델이 복잡해질수록 개별 평가에 시간이 너무 소요됨 
  * 예를 들어 최근 딥러닝에서는 수시간, 수일이 소요
* 수동 모수 최적화에서의 Trick들 
  * 전체 데이터가 아닌 부분만 사용
  * 단지 몇 번의 iteration만 해보기 (early termination of bad runs)
  * 일부 feature에 대해서만 하기
  * 고해상도가 아닌 저해상도 이미지로 만들어서 하기 
  * cross-validation fold 수를 줄이기 
* 이러한 trick은 성능 최적화와 소요시간(runtime) 사이의 tradf-off 관계에 기인한 것이다. 
* MFO는 이러한 trick을 보다 정교한 theory로서 전개하는 것을 일컫는다.


### 1.4.1 Learning Curve-based Prediction for Early Stopping

* 싹수가 노란 모수 실험 시도를 조기에 종료하는 것으로 고속화를 꾀한다. 
* 조기 종료의 기준을 학습 곡선의 외삽(extrapolation)
  * 즉 학습 초반의 경향성을 보고 (by fitting), 낮은 성능으로 saturate될 것으로 예상되는 bad run을 골라낸다. 
  * predictive termination

#### [26] Speeding up automatic hyperparameter optimization of deep neural networks by extrapolation of learning curves, 2015

* 11 parameteric function based fitting
* 그냥 BO하는 것보다 더 나은 최적화 성능 뿐만 아니라, 2,3배의 speed-up도 된다. 
* CIFAR-10에서의 SOTA 달성

#### [80]   Learning curve prediction with Baysian neural networks, 2017

* [26]에서는 특정 모두 조합에 국한해서 fitting을 해야 하고, 다른 조합에 대해서는 처음부터 fitting을 다시 해야 한다. 
* 일종의 transfer learning처럼 기존 fitting 정보를 바탕으로 더 효율적인 fitting을 하는 방법에 대한 연구 

#### [148] Freeze-Thaw Baysian optimization, 2014

* learning curve에 대한 고려를 BO 안으로통합시키는 매커니즘 연구 



### 1.4.2 Bandit-based Algrithm Selection Methods

* 성능 측정의 저신뢰 근사화(low-fidelity approx)를 기반으로 여러 알고리즘 중에서 최적의 알고리즘 선택하는 것
* successive halving과 hyperband

#### [69] Non-stochastic best arm identification and hyperparameter optimization

##### successive halving
  * 주어진 고정 예산(budget)에서 모든 알고리즘을 테스트(query)하고, 그 중 성능이 떨어지는 절반을 제거한다. 
  * 예산을 두배로 늘리고, 위 과정을 계속 반복하여 하나의 best 알고리즘을 택한다. 

![](./images/ch01/04.png )

* 이론적으로 uniform budget allocation strategy보다 우수하다. 
  * 잘 알려진 UCB, EXP3보다도 선호된다. 
* 단점 
  * suffer from budget vs number of configuration trade off
  * 전체 예산이 있을 때, 많은 조합(configuration)에게 조금씩 예산을 줄지, 적은 조합에 많은 예산을 줄지 결정
  * 각 조합에 너무 적은 예산을 주면 premature termination발생, 너무 많은 예산은 적절한 termination이 안되고 질질 끈다?

#### [90] A novel bandit-based approach to hyperparameter optimization, 2018

* Hyperband
* hedge strategy 기반 
* 다양한 조합수 기반...

* 하지만 not adapt the configuration proposal strategy

#### [33] BOHB: Robust and Efficient Hyperparameter Optimization at Scale, 2018




### 1.4.3 Adaptive Choices of Fidelities 

* 기존은 특정 fidelity를 딱 정해서 사용하는데..
* 상황의 변화에 따라 더 나은 fidelity로 적응하는(adaptive) 방벋들 

#### [147] Multi-task Bayesian Optimization, NIPS, 2013

* 각 테스크가 서로 다른 low-fidelity에 해당 
* cost-aware information-theoretic acquisition function을 바탕으로 더 나은 task로 동적으로 마구 전환한다. 
* 현실적으로는 초반에는 lowest-fidelity 기반 수행을 하다가, 점점 조금 더 비싼 fidelity로 전환한다.
* 기존 BO 대비 2x Speed up

#### [78] Fast Baysian hyperparameter optimization on large datasets, 2017

* [147]에서는 특정 set of fidelity를 사전에 정의(specification)해야 하고, 그 수도 5~6개 정도에 그친다. 
* fidelity를 일종의 continous한 것으로 하면(ex. % of dataset) set 크기가 무한이 되며, 이를 다루는 kernel 정의 
* 기존 BO 대비 10-100 speed up

#### [74] BOCA

* UCB + BO + continous multi-fidelity





### 저자들의 실용적 제안

* multiple fidelity가 있는 상황이라면 BOHB를 써라
* muliple fidelity가 없는 상황이라면 
  * 모수가 모두 실수기반, 함수 평가를 수십번 정도밖에 못하는 상황이라면 Spearmint [140]
  * 모수가 많고, 조건적이라면 SMAC, TPE
  * 모수가 실수이고, 함수 평가가 다소 간단하다면, CMA-ES

## 1.6 Open Problem and Future Research Direction

### Benchmark and Compatibility

* 객관적인 장단점 파악을 위한 커뮤니티가 인정하는 벤치마크가 필요하다. 
  * COCO platform 이 가장 유명, HPOLib도 있다. 
  * BBOB Challenge (Black-box optimization Benchmarking)
* 모두가 동의하는 metric이 필요
  * 기준 데이터셋을 어떤 것으로 할 것인가 (valid v.s test)
    * valid set은 noise없이 고립(isolation)되서 성능 측정 가능하나 overfit된 추정이라는 한계
  * 최종 성능 측정을 정해진 몇번의 함수 평가 시점에 할 것인지 아니면 정해진 시간 후에 할 것인지
    * 후자는 다른 초모수 조합 평가시의 시간차가 있다는 점을 고려한 것이므로 실용적이다. 
    * 반면 전자는 실험 재현성이 탁월하다. 

### Gradient-based Optimization

* 어떤 것들은(SVM) Black-box optimization과는 다르게 초모수에 대한 gradient를 얻을 수도 있다. 
* hypergradient vector

#### [99] Gradient-based Hyperparameter Optimization through Reversible Learning


### Scalability

* 여전히 광대한 configuration space와 비싼 개별 함수 평가로 인해 scalable하지 않다. 
* 예를 들어서 아직까지도 ImageNet challenge에 대해서는 HPO를 못하고 있다. 
* 분산화된 HPO 개념 필요하다. 
  * ex) parallel baysian optimization for neural network ( extended to hundreds of worker)
* 더 정교한 것 기대 

### Overfit and Generalization


